In [4]:
def Validation(n_fold,X,Y):
    list_train_fold = []
    list_val_fold = []
    list_train = []
    Number = X.shape[0]//n_fold
    for i in range(X.shape[0]):
        list_train.append(i)
    
    for i in range(n_fold)[::-1]:
        list_val = []
        if i==n_fold-1:
            for j in range(Number*i,X.shape[0]):
                list_val.append(j)
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
            
        
        if i != n_fold-1: 
            for j in range(Number*i,Number*(i+1)):
                list_val.append(j)
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
        
    return list_train_fold, list_val_fold

In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv("Data/PPI.csv")
data.drop(['string'], axis='columns', inplace=True)
data1 = pd.read_csv("Data/Table_S6_GDSC_Drug_response_IC50.csv")
data1.drop(['Unnamed: 0'], axis='columns', inplace=True)
data2 = pd.read_csv("Data/EXP.csv")
data2.drop(['Unnamed: 0'], axis='columns', inplace=True)
data2=np.array(data2)

scaler = MinMaxScaler()
scaler.fit(data1)
data1 = scaler.transform(data1)

from sklearn.model_selection import train_test_split
   
train_data_split, test_data_split, train_labels_split, test_labels_split = train_test_split(data2, data1, 
                                                      test_size=0.25, shuffle=True, random_state=20)


X_train = np.array(train_data_split[:]).astype(np.float32)
X_test = np.array(test_data_split[:]).astype(np.float32)
Y_train = np.array(train_labels_split).astype(np.float32)
Y_test = np.array(test_labels_split[:]).astype(np.float32)



In [18]:
# KernelRidge
from sklearn.kernel_ridge import KernelRidge
import numpy as np

n_fold = 3
for cv in range(n_fold):
    model = KernelRidge()
    Y_pred = np.zeros([X_test.shape[0], Y_train.shape[1]])
    for i in range(Y_train.shape[1]):
        y_train = Y_train[:, i]
        x_train = X_train[~np.isnan(y_train)]
        y_train = y_train[~np.isnan(y_train)]
        
        train_fold, val_fold = Validation(n_fold, x_train,y_train)
        
        model.fit(x_train[train_fold[cv]], y_train[train_fold[cv]])
        Y_pred[:, i] = model.predict(X_test)
    #np.savez(('Result/KRR_CV_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

In [3]:
data1

array([[0.52818121, 0.40454562, 0.58735693, ..., 0.44591466, 0.46112163,
        0.81931499],
       [0.51166003, 0.72085953, 0.52139631, ..., 0.28976016, 0.53859058,
        0.74478388],
       [0.56139802, 0.50876953, 0.17590553, ..., 0.11895323, 0.27817899,
        0.61092248],
       ...,
       [0.58204285, 0.53085672,        nan, ..., 0.28845841, 0.28036646,
        0.76446601],
       [0.58808006, 0.757052  , 0.4865119 , ..., 0.31618119, 0.50524049,
        0.64475449],
       [0.52137608, 0.75290445, 0.87674484, ..., 0.61302454, 0.55428691,
        0.92063257]])

In [16]:
a,b = Validation(2, x_train,y_train)
a

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [ ]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

n_fold = 1
for cv in range(n_fold):
    model = RandomForestRegressor()
    Y_pred = np.zeros([X_test.shape[0], Y_train.shape[1]])
    print("CV :",cv)
    for i in range(Y_train.shape[1]):
        y_train = Y_train[:, i]
        x_train = X_train[~np.isnan(y_train)]
        y_train = y_train[~np.isnan(y_train)]
        
        val_fold, train_fold = Validation(n_fold, x_train,y_train)
        model.fit(x_train[train_fold[cv]], y_train[train_fold[cv]])
        Y_pred[:, i] = model.predict(X_test)
        print(i, end = ' ')
    print("CV fold",cv,"종료")
    np.savez(('Result/Fold1_RF_CV_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)
    

CV : 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 

In [ ]:
# MLPRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np

n_fold = 3
for cv in range(n_fold):
    model = MLPRegressor(hidden_layer_sizes = (1000,100))
    Y_pred = np.zeros([X_test.shape[0], Y_train.shape[1]])
    print("CV :",cv)
    for i in range(Y_train.shape[1]):
        y_train = Y_train[:, i]
        x_train = X_train[~np.isnan(y_train)]
        y_train = y_train[~np.isnan(y_train)]
        
        train_fold, val_fold = Validation(n_fold, x_train,y_train)
        
        model.fit(x_train[train_fold[cv]], y_train[train_fold[cv]])
        Y_pred[:, i] = model.predict(X_test)
        print(i, end = ' ')
    print("CV fold",cv,"종료")
    np.savez(('Result/MLP_CV_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

CV : 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 

In [ ]:
# BaggingRegressor
from sklearn.ensemble import BaggingRegressor

n_fold = 3
for cv in range(n_fold):
    model = BaggingRegressor()
    Y_pred = np.zeros([X_test.shape[0], Y_train.shape[1]])
    print("CV :",cv)
    for i in range(Y_train.shape[1]):
        y_train = Y_train[:, i]
        x_train = X_train[~np.isnan(y_train)]
        y_train = y_train[~np.isnan(y_train)]
        
        train_fold, val_fold = Validation(n_fold, x_train,y_train)
        
        model.fit(x_train[train_fold[cv]], y_train[train_fold[cv]])
        Y_pred[:, i] = model.predict(X_test)
        print(i, end = ' ')
    print("CV fold",cv,"종료")
    np.savez(('Result/BR_CV_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

CV : 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 CV fold 0 종료
CV : 1
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 9

In [6]:
from KRL.KRL import KRL 
n_fold = 3
for cv in range(n_fold):
    Y_pred = np.zeros([X_test.shape[0], Y_train.shape[1]])
    print("CV :",cv)
    list_train, list_val = Validation(n_fold, X_train, Y_train)
    
    X_train_cv = np.array(X_train)[list_train[cv]]
    X_test_cv = np.array(X_test)
    Y_train_cv = np.array(Y_train)[list_train[cv]]
    Y_test_cv = np.array(Y_test)

    Y_pred = KRL(X_train_cv,Y_train_cv,X_test_cv,10,1,None,4,True)

    np.savez(('Result/KRL_CV_{}'.format(cv)), Y_true=Y_test_cv, Y_pred=Y_pred)
    print("CV fold",cv,"종료")

CV : 0
Epsilon iter 1: 146050.9741821429
Epsilon iter 2: 14054.011579337446
Epsilon iter 3: 3589.3206083312807
Epsilon iter 4: 2160.429423748144
Epsilon iter 5: 1714.94278918821
Epsilon iter 6: 1554.927148289219
Epsilon iter 7: 1494.2926122752788
Epsilon iter 8: 596.7103973967387
Epsilon iter 9: 445.5600661435095
Epsilon iter 10: 185.83909766642824
Epsilon iter 11: 82.91673043523576
Epsilon iter 12: 11.081234730580604
Epsilon iter 13: 0.48827280455293476
Epsilon iter 14: 0.2754807223402054
Epsilon iter 15: 0.007549467820240352
Epsilon iter 16: -0.2079804901833313
CV fold 0 종료
CV : 1
Epsilon iter 1: 148344.07727228565
Epsilon iter 2: 13892.878988436121
Epsilon iter 3: 3495.376682437951
Epsilon iter 4: 2132.0113590581395
Epsilon iter 5: 1708.55985838862
Epsilon iter 6: 1560.5144439175374
Epsilon iter 7: 1510.5712921204408
Epsilon iter 8: 1036.585324637943
Epsilon iter 9: 375.38884822020714
Epsilon iter 10: 372.44342347805497
Epsilon iter 11: 193.9533657674059
Epsilon iter 12: 136.6220769